# 날씨 데이터 크롤링

출처: https://www.data.go.kr/  
ASOS 종관기상관측 데이터 활용

# URL 설정

In [1]:
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus
import urllib
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
import datetime

In [2]:
key= 'API key'
url="http://data.kma.go.kr/apiData/getData"


# 일별데이터 출력

### api 설명서에 있는 활용가이드에서 요청/응답 메세지 예제를 확인한다

### urlparse, parse_qs를 통해 파라미터로 나눠준다

In [3]:
from urllib.parse import urlparse, parse_qs
url = urlparse("http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey=13nWfyvckU06jS8j02cgKEde9gF7%2B9BZ%2BOFcpgpuu%2FnNRbR64QYGlNEzN7b%2F9D4RI3uFKkJ97YT9mf5X3FMIUA%3D%3D&numOfRows=10&pageNo=1&dataCd=ASOS&dateCd=DAY&startDt=20100101&endDt=20100102&stnIds=108")
par_dic = parse_qs(url.query)
for key in par_dic:
    par_dic[key] = par_dic[key][0]
par_dic

{'serviceKey': '13nWfyvckU06jS8j02cgKEde9gF7+9BZ+OFcpgpuu/nNRbR64QYGlNEzN7b/9D4RI3uFKkJ97YT9mf5X3FMIUA==',
 'numOfRows': '10',
 'pageNo': '1',
 'dataCd': 'ASOS',
 'dateCd': 'DAY',
 'startDt': '20100101',
 'endDt': '20100102',
 'stnIds': '108'}

### 공공데이터에서 어제까지의 기상정보 데이터를 json형식으로 요청해 가져온다

In [4]:
def get_daily_data(start_date):
    start_date=start_date 
    end_date=str(datetime.date.today()-datetime.timedelta(1)).replace('-','')
    url='http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'

    
    params={'serviceKey': unquote('API key'),
            'numOfRows': '200',
            'pageNo': '1',
            'dataCd': 'ASOS',
            'dateCd': 'DAY',
            'startDt': start_date,
            'endDt': end_date,
            'stnIds': '108',
            'dataType':'JSON'
           }

    resp = requests.get(url, params=params)
    result= json.loads(resp.text)
    return result

In [5]:
def get_hourly_data(start_date, start_hour,end_date,end_hour):
    start_date=start_date
    start_hour=start_hour
    end_date=end_date
    end_hour=end_hour
    url='http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'

    
    params={'serviceKey': 'API key',
     'numOfRows': '10',
     'pageNo': '1',
     'dataCd': 'ASOS',
     'dateCd': 'HR',
     'stnIds': '108',
     'schListCnt': '10',
     'endDt': end_date,
     'endHh': end_hour,
     'startHh': start_hour,
     'startDt': start_date,
     'dataType': 'JSON'
           }

    resp = requests.get(url, params=params)
    #result= json.loads(resp.text)

    return resp.text

### 평균온도에 따른 옷차림을 보여주는 함수를 만든다

In [6]:
def what_to_wear(high,low):
    avg = (int(high[0])+int(low[0]))/2
    if avg >= 28:
            clothes = '나시티 / 반바지 / 민소매 / 원피스'
    elif avg >= 23:
        clothes = '반팔 / 얇은 셔츠 / 얇은 긴팔 / 반바지 / 면바지'
    elif avg >= 20:
        clothes = '긴팔티 / 가디건 / 후드티 / 면바지 / 슬랙스 / 청바지'
    elif avg >= 17:
        clothes = '니트 / 가디건 / 후드티 / 맨투맨 / 면바지 / 슬랙스 / 청바지'
    elif avg >= 12:
        clothes = '자켓 / 셔츠 / 가디건 / 간절기 외투'
    elif avg >= 9:
        clothes = '트렌치 코트 / 간절기 외투 / 여러겹 껴입기'
    elif avg >= 6:
        clothes = '코트 / 가죽자켓'
    else:
        clothes = '겨울 옷(야상, 패딩, 목도리 등등 다)'
        
    return clothes

### 날씨 json 데이터에서 필요한 부분을 추출해 데이터 프레임 형태로 저장한다

In [10]:
def data_to_DataFrame(result):
    total_data=result['response']['body']['items']['item']
    
    result = {'날짜':[],
              '최저기온':[],
              '최고기온':[],
              '비':[],
              '강수량':[],
              '옷차림':[],
              '날씨':[]
             }
    for data in total_data:
        result['날짜'].append(data['tm'])
        result['최저기온'].append(data['minTa'])
        result['최고기온'].append(data['maxTa'])
        avg = (data['minTa']+data['maxTa'])/2
        if data['sumRn']>0:
            result['비'].append('O')
            result['날씨'].append('비')
        else:
            if avg<10:
                result['날씨'].append('추위')
            elif avg<20:
                result['날씨'].append('쌀쌀')
            else:
                result['날씨'].append('더위')
            result['비'].append('X')
        result['강수량'].append(data['sumRn'])
        clothes = what_to_wear(result['최저기온'],result['최고기온'])
        result['옷차림'].append(clothes)
        
    df=pd.DataFrame(result)
    
    return df

### 공공데이터에선 당일 날씨를 수집할 수 없다 

### 당일 날씨정보를 추가하기 위해 웨더아이 사이트의 xml정보를 수집하고 기존 데이터에 추가한다

In [11]:
def append_today(data):
    url = 'https://www.weatheri.co.kr/forecast/forecast01.php'
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, from_encoding='utf-8')
    attrs={'width':'615',
      'border':'0',
      'cellpadding':'1',
      'cellspacing':'1',
       'bgcolor':'#BCBFC2'
      }
    inner_table = soup.find('table',attrs=attrs)
    date = inner_table.select('td b')[0].text
    date = '2020-{0}-{1}'.format(date.split()[0][:2],date.split()[1][:2])
    high = inner_table.select('table tr td b font')[0].text
    high = high.replace('˚C','')
    low = inner_table.select('table tr td b font')[0].text
    low = low.replace('˚C','')
    sum_rain=inner_table.select('table tr td font')[-1].text
    avg = (int(low)+int(high))/2
    if sum_rain !='- mm':
        rain='O'
        weather = '비'
        sum_rain = sum_rain.split()[0]
        
    else:
        rain='X'
        sum_rain = 0
        if avg<10:
            weather = '추위'
        elif avg<20:
            weather = '쌀쌀'
        else:
            weather = '더위'
    clothes = what_to_wear(high,low)
    
        
    new_data=pd.DataFrame({'날짜':[date],
             '최저기온':[low],
             '최고기온':[high],
             '비':[rain],
             '강수량':[sum_rain],
             '옷차림':[clothes],
             '날씨':[weather]})
#     print(sum_rain)
    data = data.append(new_data,ignore_index=True)
#     print(data)
    return data

### 활용 예시))

In [12]:
past_data=get_daily_data('20200501')
until_today=data_to_DataFrame(past_data)
final_data=append_today(until_today)

0
            날짜  최저기온  최고기온  비   강수량                                옷차림  날씨
0   2020-05-01  16.4  26.2  X   0.0  긴팔티 / 가디건 / 후드티 / 면바지 / 슬랙스 / 청바지  더위
1   2020-05-02    18  23.9  X   0.0  긴팔티 / 가디건 / 후드티 / 면바지 / 슬랙스 / 청바지  더위
2   2020-05-03    17  27.4  X   0.0  긴팔티 / 가디건 / 후드티 / 면바지 / 슬랙스 / 청바지  더위
3   2020-05-04  14.8  25.3  X   0.0  긴팔티 / 가디건 / 후드티 / 면바지 / 슬랙스 / 청바지  더위
4   2020-05-05  13.1  19.3  X   0.0  긴팔티 / 가디건 / 후드티 / 면바지 / 슬랙스 / 청바지  쌀쌀
5   2020-05-06  11.1  27.6  X   0.0  긴팔티 / 가디건 / 후드티 / 면바지 / 슬랙스 / 청바지  쌀쌀
6   2020-05-07  14.9  26.5  X   0.0  긴팔티 / 가디건 / 후드티 / 면바지 / 슬랙스 / 청바지  더위
7   2020-05-08  14.3  27.3  O   2.3  긴팔티 / 가디건 / 후드티 / 면바지 / 슬랙스 / 청바지   비
8   2020-05-09  12.7  16.8  O  24.4  긴팔티 / 가디건 / 후드티 / 면바지 / 슬랙스 / 청바지   비
9   2020-05-10  12.8  16.6  O   1.3  긴팔티 / 가디건 / 후드티 / 면바지 / 슬랙스 / 청바지   비
10  2020-05-11  12.4    22  O   3.4  긴팔티 / 가디건 / 후드티 / 면바지 / 슬랙스 / 청바지   비
11  2020-05-12  10.5  18.9  O   0.7  긴팔티 / 가디건 / 후드티 / 면바지 / 슬랙스 / 청바지   비
12  2020-05-13  10.2  2

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\ProgramData\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [13]:
final_data.to_csv('weather_crawling.csv',encoding='utf-8', index=False)
print('=====END Weather Crawling=====')

=====END Weather Crawling=====
